In [3]:
# Importação de bibliotecas necessárias para diversas funções
from bs4 import BeautifulSoup  # Utilizado para parsing de documentos HTML e XML
import requests  # Utilizado para fazer requisições HTTP
import html5lib  # Parser para documentos HTML e XML
import os  # Utilizado para interagir com o sistema operacional
from dateutil.relativedelta import relativedelta  # Utilizado para cálculos de diferenças de datas
import pandas as pd  # Utilizado para manipulação e análise de dados
import numpy as np  # Utilizado para cálculos numéricos
import pyarrow.parquet as pq  # Utilizado para leitura e escrita de dados em formato Parquet
import pyarrow as pa  # Utilizado para manipulação de dados com Apache Arrow
import pyarrow.dataset as ds  # Utilizado para manipulação de datasets com Apache Arrow
import pyarrow.compute as pc  # Utilizado para computação de funções com Apache Arrow
from urllib.request import urlopen  # Utilizado para abrir URLs
from io import BytesIO  # Utilizado para manipulação de fluxos de bytes
from zipfile import ZipFile  # Utilizado para manipulação de arquivos zip
from datetime import datetime  # Utilizado para manipulação de datas e horas
import warnings  # Utilizado para gerenciar avisos em Python

# Ignorar avisos de RankingWarning do numpy
warnings.simplefilter('ignore', np.RankWarning)

# Configuração do nível de logging para INFO
import logging
logging.getLogger().setLevel(logging.INFO)

# Desabilitar a verificação SSL para requests ao definir a variável de ambiente 'verify' como "False"
# Nota: Desabilitar a verificação SSL não é recomendado para produção devido a riscos de segurança.
os.environ['verify'] = "False"

# Faz uma requisição HTTP para o Google com a verificação SSL desabilitada
requests.get('https://www.google.com', verify=False)

# Definição das URLs para dados históricos de fundos de investimento disponibilizados pela CVM
URL_HIS_FI_NEW_YEARS = 'https://dados.cvm.gov.br/dados/FI/DOC/CDA/DADOS/'
URL_HIS_FI_OLD_YEARS = 'https://dados.cvm.gov.br/dados/FI/DOC/CDA/DADOS/HIST/'

c:\Users\Rodrigo\Python\CodigosTrabalho\venv\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
# Ler um dataset existente do diretório especificado usando Apache Arrow
dataset = ds.dataset(r'C:\Users\Rodrigo\Python\PythonCVM\CarteirasCVMParquet')

# Fazer requisição HTTP para obter dados dos anos recentes de fundos de investimento da CVM
request_new_years = requests.get(URL_HIS_FI_NEW_YEARS, verify=False)

# Fazer requisição HTTP para obter dados dos anos antigos de fundos de investimento da CVM
request_old_years = requests.get(URL_HIS_FI_OLD_YEARS, verify=False)

# Analisar o conteúdo HTML da resposta HTTP para os dados recentes usando BeautifulSoup
soup_new_years = BeautifulSoup(request_new_years.content, features='html5lib')

# Analisar o conteúdo HTML da resposta HTTP para os dados antigos usando BeautifulSoup
soup_old_years = BeautifulSoup(request_old_years.content, features='html5lib')

c:\Users\Rodrigo\Python\CodigosTrabalho\venv\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dados.cvm.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\Rodrigo\Python\CodigosTrabalho\venv\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dados.cvm.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [ ]:
# Definir a data de início do histórico de dados
DATE_INI = '2005-01-01'

# Calcular a data atual adicionando um mês à data de hoje e formatar no formato 'MM-YYYY'
CURRENT_DATE = pd.to_datetime(datetime.today() + relativedelta(months=1)).strftime('%m-%Y')

# Buscar dados na CVM
# Cria uma faixa de datas mensais desde a data de início até a data atual e formata como 'YYYYMM'
date = pd.date_range(start=str(DATE_INI), end=str(CURRENT_DATE), freq='M').strftime('%Y%m')

# Cria uma lista dos anos únicos dentro da faixa de datas mensais desde a data de início até a data atual
years = list(pd.date_range(start=str(DATE_INI), end=str(CURRENT_DATE), freq='M').strftime('%Y').unique())

In [ ]:
# Inicializa uma lista para armazenar os anos disponíveis na CVM
years_cvm = []

# Itera sobre todos os links encontrados na página HTML de dados antigos
for link in soup_old_years.find_all('a'):
    # Verifica se o comprimento do atributo 'href' do link é igual a 15
    if len(link.get('href')) == 15:
        # Extrai os anos do atributo 'href' e adiciona à lista 'years_cvm'
        years_cvm.append(link.get('href')[7:11])

# Filtra os anos para incluir apenas aqueles maiores ou iguais ao menor ano na lista 'years'
year_filter = list(filter(lambda score: score >= min(years), years_cvm))

# Exibe a lista de anos filtrados
year_filter

In [ ]:
# Função para mapear tipos de dados do Pandas para tipos de dados do PyArrow
def pandas_to_pyarrow_dtype(dtype):
    # Verifica se o tipo de dados do Pandas é um tipo inteiro
    if pd.api.types.is_integer_dtype(dtype):
        return pa.int64()  # Retorna o tipo de dados inteiro de 64 bits do PyArrow
    # Verifica se o tipo de dados do Pandas é um tipo float
    elif pd.api.types.is_float_dtype(dtype):
        return pa.float64()  # Retorna o tipo de dados float de 64 bits do PyArrow
    # Verifica se o tipo de dados do Pandas é uma string
    elif pd.api.types.is_string_dtype(dtype):
        return pa.string()  # Retorna o tipo de dados string do PyArrow
    # Verifica se o tipo de dados do Pandas é um tipo datetime
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return pa.timestamp('ns')  # Retorna o tipo de dados timestamp (nanosegundos) do PyArrow
    # Lança uma exceção se o tipo de dados do Pandas não for suportado
    else:
        raise ValueError(f"tipo de dados Pandas não suportado: {dtype}")

In [ ]:
#Formatação tipo de dado DataFrame
def trataFormato(df_inf_raw):
    df_inf_raw['QT_VENDA_NEGOC'] = df_inf_raw['QT_VENDA_NEGOC'].astype(float)
    df_inf_raw['VL_VENDA_NEGOC'] = df_inf_raw['VL_VENDA_NEGOC'].astype(float)
    df_inf_raw['QT_AQUIS_NEGOC'] = df_inf_raw['QT_AQUIS_NEGOC'].astype(float)
    df_inf_raw['VL_AQUIS_NEGOC'] = df_inf_raw['VL_AQUIS_NEGOC'].astype(float)
    df_inf_raw['QT_POS_FINAL'] = df_inf_raw['QT_POS_FINAL'].astype(float)
    df_inf_raw['VL_MERC_POS_FINAL'] = df_inf_raw['VL_MERC_POS_FINAL'].astype(float)
    df_inf_raw['VL_CUSTO_POS_FINAL'] = df_inf_raw['VL_CUSTO_POS_FINAL'].astype(float)
    df_inf_raw['DT_VENC'] = df_inf_raw['DT_VENC'].astype(pd.StringDtype())
    try:
        df_inf_raw['INVEST_COLETIVO_GESTOR'] = df_inf_raw['INVEST_COLETIVO_GESTOR'].astype(pd.StringDtype())
    except Exception:
        pass
    try:
        df_inf_raw['INVEST_COLETIVO'] = df_inf_raw['INVEST_COLETIVO'].astype(pd.StringDtype())
    except Exception:
        pass
    try:
        df_inf_raw['PR_INDEXADOR_POSFX'] = df_inf_raw['PR_INDEXADOR_POSFX'].astype(float)
    except Exception:
        pass
    try:
        df_inf_raw['PR_CUPOM_POSFX'] = df_inf_raw['PR_CUPOM_POSFX'].astype(float)
    except Exception:
        pass
    try:
        df_inf_raw['PR_TAXA_PREFX'] = df_inf_raw['PR_TAXA_PREFX'].astype(float)
    except Exception:
        pass
    try:
        df_inf_raw['QT_ATIVO_EXTERIOR'] = df_inf_raw['QT_ATIVO_EXTERIOR'].astype(float)
    except Exception:
        pass
    try:
        df_inf_raw['VL_ATIVO_EXTERIOR'] = df_inf_raw['VL_ATIVO_EXTERIOR'].astype(float)
    except Exception:
        pass
    try:
        df_inf_raw['CD_BV_MERC'] = df_inf_raw['CD_BV_MERC'].astype(pd.StringDtype())
    except Exception:
        pass
    try:
        df_inf_raw['CD_ATIVO_BV_MERC'] = df_inf_raw['CD_ATIVO_BV_MERC'].astype(pd.StringDtype())
    except Exception:
        pass
    try:
        df_inf_raw['AG_RISCO'] = df_inf_raw['AG_RISCO'].astype(pd.StringDtype())
    except Exception:
        pass
    try:
        df_inf_raw['DT_RISCO'] = df_inf_raw['DT_RISCO'].astype(pd.StringDtype())
    except Exception:
        pass
    return df_inf_raw

In [ ]:
#Baixando carteiras até o ano que a CVM passa abrir por nès encoding - 'utf-8'
df_inf_raw = pd.DataFrame()
logging.info('Anos Pasta HIST: {}. '.format(year_filter))
for year in year_filter:
    url = 'https://dados.cvm.gov.br/dados/FI/DOC/CDA/DADOS/HIST/cda_fi_'+year+'.zip'
    logging.info('Obtendo dados do Ano: {}.'.format(year))
    with ZipFile(BytesIO(urlopen(url).read())) as myzip:
        num = 1
        while num < 20:
            try:
                with myzip.open('cda_fi_BLC_' + str(num) + '_' + year+'.csv') as myfile:
                    df_inf_raw = pd.concat([df_inf_raw, pd.read_csv(myfile, encoding = 'latin1', engine = 'python', on_bad_lines='skip', sep=';',decimal=',')])
            except Exception:
                pass
            num +=1
    
    df_inf_raw = trataFormato(df_inf_raw)

    #criar o schema PyArrow a partir dos dtypes do DataFrame
    schema = pa.schema([(col, pandas_to_pyarrow_dtype(df_inf_raw[col].dtype)) for col in df_inf_raw.columns])
    
    #Convertendo os dados em tabela pyArrow
    novos_dados_table = pa.Table.from_pandas(df_inf_raw, schema=schema)
    
    #Escrevendo os novos dados em um novo arquivo Parquet no dataset
    pq.write_to_dataset(novos_dados_table,root_path=r'C:\Users\Rodrigo\Python\PythonCVM\CarteirasCVMParquet')
    novos_dados_table = novos_dados_table[0:0]
    df_inf_raw = df_inf_raw[0:0]



In [ ]:
#Baixando carteiras mensais até faltar 6 meses
datas = []
df_inf_raw = pd.DataFrame()
for link in soup_new_years.find_all('a'):
     if len(link.get('href')) == 17:
         datas.append(link.get('href')[7:13])
logging.info('Meses Recentes: {}. '.format(datas))
meses_cvm = len(datas)
i=0
for dt in datas:
    print(dt)
    if i< meses_cvm - 6:
        url = 'https://dados.cvm.gov.br/dados/FI/DOC/CDA/DADOS/cda_fi_'+dt+'.zip'
        with ZipFile(BytesIO(urlopen(url).read())) as myzip:
            num = 1
            while num < 20:
                try:
                    with myzip.open('cda_fi_BLC_' + str(num) + '_' + dt + '.csv') as myfile:
                        df_inf_raw = pd.concat([df_inf_raw, pd.read_csv(myfile, encoding = 'latin1', sep=";" ,decimal=",")])
                except Exception:
                    pass
                num+=1
        
        df_inf_raw['QT_VENDA_NEGOC'] = df_inf_raw['QT_VENDA_NEGOC'].astype(float)
        df_inf_raw['VL_VENDA_NEGOC'] = df_inf_raw['VL_VENDA_NEGOC'].astype(float)
        df_inf_raw['QT_AQUIS_NEGOC'] = df_inf_raw['QT_AQUIS_NEGOC'].astype(float)
        df_inf_raw['VL_AQUIS_NEGOC'] = df_inf_raw['VL_AQUIS_NEGOC'].astype(float)
        df_inf_raw['QT_POS_FINAL'] = df_inf_raw['QT_POS_FINAL'].astype(float)
        df_inf_raw['VL_MERC_POS_FINAL'] = df_inf_raw['VL_MERC_POS_FINAL'].astype(float)
        df_inf_raw['VL_CUSTO_POS_FINAL'] = df_inf_raw['VL_CUSTO_POS_FINAL'].astype(float)
        df_inf_raw['DT_VENC'] = df_inf_raw['DT_VENC'].astype(pd.StringDtype())
        try:
            df_inf_raw['INVEST_COLETIVO_GESTOR'] = df_inf_raw['INVEST_COLETIVO_GESTOR'].astype(pd.StringDtype())
        except Exception:
            pass
        try:
            df_inf_raw['INVEST_COLETIVO'] = df_inf_raw['INVEST_COLETIVO'].astype(pd.StringDtype())
        except Exception:
            pass
        try:
            df_inf_raw['PR_INDEXADOR_POSFX'] = df_inf_raw['PR_INDEXADOR_POSFX'].astype(float)
        except Exception:
            pass
        try:
            df_inf_raw['PR_CUPOM_POSFX'] = df_inf_raw['PR_CUPOM_POSFX'].astype(float)
        except Exception:
            pass
        try:
            df_inf_raw['PR_TAXA_PREFX'] = df_inf_raw['PR_TAXA_PREFX'].astype(float)
        except Exception:
            pass
        try:
            df_inf_raw['QT_ATIVO_EXTERIOR'] = df_inf_raw['QT_ATIVO_EXTERIOR'].astype(float)
        except Exception:
            pass
        try:
            df_inf_raw['VL_ATIVO_EXTERIOR'] = df_inf_raw['VL_ATIVO_EXTERIOR'].astype(float)
        except Exception:
            pass
        try:
            df_inf_raw['CD_BV_MERC'] = df_inf_raw['CD_BV_MERC'].astype(pd.StringDtype())
        except Exception:
            pass
        try:
            df_inf_raw['CD_ATIVO_BV_MERC'] = df_inf_raw['CD_ATIVO_BV_MERC'].astype(pd.StringDtype())
        except Exception:
            pass
        try:
            df_inf_raw['AG_RISCO'] = df_inf_raw['AG_RISCO'].astype(pd.StringDtype())
        except Exception:
            pass
        try:
            df_inf_raw['DT_RISCO'] = df_inf_raw['DT_RISCO'].astype(pd.StringDtype())
        except Exception:
            pass
        try:
            df_inf_raw['GRAU_RISCO'] = df_inf_raw['GRAU_RISCO'].astype(pd.StringDtype())
        except Exception:
            pass
        try:
            df_inf_raw['DS_ATIVO_EXTERIOR'] = df_inf_raw['DS_ATIVO_EXTERIOR'].astype(pd.StringDtype())
        except Exception:
            pass   

        #criar o schema PyArrow a partir dos dtypes do Datarrame
        schema = pa.schema([(col, pandas_to_pyarrow_dtype(df_inf_raw[col].dtype)) for col in df_inf_raw.columns])
        
        #Convertendo os dados em tabela pyArrow
        novos_dados_table = pa.Table.from_pandas(df_inf_raw, schema=schema)
        
        #Escrevendo os novos dados em um novo arquivo Parquet no dataset
        pq.write_to_dataset (novos_dados_table, root_path=r'C:\Users\Rodrigo\Python\PythonCVM\CarteirasCVMParquet')
        novos_dados_table = novos_dados_table[0:0]
        df_inf_raw = df_inf_raw[0:0]
        i=i+1         


In [ ]:
DATE_INI = '2005-01' #Data de início do Histórico
CURRENT_DATE = pd.to_datetime(datetime.today() + relativedelta(months=1)).strftime('%m-%Y')

##Buscar Dados na CVM
date = pd.date_range(start=str(DATE_INI), end=str(CURRENT_DATE), freq='M').strftime('%Y%m')
years = list(pd.date_range(start=str(DATE_INI), end=str(CURRENT_DATE), freq='M').strftime('%Y').unique())
years_cvm = []
for link in soup_old_years.find_all('a'):
    if len(link.get('href')) == 15:
        years_cvm.append(link.get('href')[7:11])
year_filter = (list(filter(lambda score: score >= min(years), years_cvm)))
year_filter

#Baixando PL dos fundos até o ano que a CVM passa abrir por mês
encoding = 'utf-8'
df_int_raw = pd.DataFrame()
logging.info('Anos Pasta HIST: {}. '.format(year_filter))
for year in year_filter:
    url = 'https://dados.cvm.gov.br/dados/FI/DOC/CDA/DADOS/HIST/cda_fi_'+year+'.zip'
    logging.info('Obtendo dados do Ano: {}.'.format(year))
    with ZipFile(BytesIO(urlopen(url).read())) as myzip:
        with myzip.open('cda_fi_PL_'+year+'.csv') as myfile:
            df_inf_raw = pd.concat([df_inf_raw, pd.read_csv(myfile, encoding = 'latin1', engine = 'python', on_bad_lines='skip', sep=';',decimal=',')])
    
    df_inf_raw['VL_PATRIM_LIQ'] = df_inf_raw['VL_PATRIM_LIQ'].astype(float)
    
    #criar o schema PyArrow a partir dos dtypes do bataFrame
    schema = pa.schema([(col, pandas_to_pyarrow_dtype(df_inf_raw[col].dtype)) for col in df_inf_raw.columns])
    
    #Convertendo os dados em tabela pyArrow
    novos_dados_table = pa.Table.from_pandas(df_inf_raw, schema=schema)

    #Escrevendo os novos dados em um novo arquivo Parquet no dataset
    pq.write_to_dataset(novos_dados_table, root_path=r'C:\Users\Rodrigo\Python\PythonCVM\PLsCVMParquet')
    novos_dados_table = novos_dados_table[0:0]
    df_inf_raw = df_inf_raw[0:0]

In [ ]:
#Baixando PLs mensais até faltar 6 meses
datas = []
df_inf_raw = pd.DataFrame()
for link in soup_new_years.find_all('a'):

    if len(link.get('href')) == 17:
        datas.append(link.get('href')[7:13])
logging.info('Meses Recentes: {}. '.format(datas))
meses_cvm = len(datas)
i=0
for dt in datas:
    if i < meses_cvm - 6:
        url = 'https://dados.cvm.gov.br/dados/FI/DOC/CDA/DADOS/cda_fi_'+dt+'.zip'
        with ZipFile(BytesIO(urlopen(url).read())) as myzip:
            with myzip.open('cda_fi_PL_'+dt+'.csv') as myfile:
                df_inf_raw = pd.concat([df_inf_raw, pd.read_csv(myfile, encoding = 'latin1', sep=";",decimal=",")])
        
        df_inf_raw['VL_PATRIM LIQ'] = df_inf_raw[ 'VL_PATRIM_LIQ'].astype(float)
        
        #criar o schema pyArrow a pandas a partir dos dtypes da DataFrame
        schema = pa.schema([(col,pandas_to_pyarrow_dtype(df_inf_raw[col].dtype)) for col in df_inf_raw.columns])

        #Convertendo os dados em tabela pyArrow
        novos_dados_table = pa.Table.from_pandas(df_inf_raw, schema=schema)
        
        #Escrevendo os novos dados em um novo arquivo Parquet no dataset
        pq.write_to_dataset(novos_dados_table,root_path=r'C:\Users\Rodrigo\Python\PythonCVM\PLsCVMParquet')
        novos_dados_table = novos_dados_table[0:0]
        df_inf_raw = df_inf_raw[0:0]
        i=i+1

In [ ]:
filter_expression = ((ds.field('CNPJ_FUNDO') == '18.255.230/0001-67') | (ds.field('CNPJ_FUNDO') == '18.272.304/0001-73') | (ds.field('CNPJ_FUNDO') == '05.471.940/0001-82') | (ds.field('CNPJ_FUNDO') == '05.464.917/0001-60') | 
(ds.field('CNPJ_FUNDO') == '05.464.917/0001-60') | (ds.field('CNPJ_FUNDO') == '05.464.914/0001-27') | (ds.field('CNPJ_FUNDO') == '05.464.914/0001-27') | (ds.field('CNPJ_FUNDO') == '05.464.910/0001-49') |
(ds.field('CNPJ_FUNDO') == '09.613.212/0001-19') | (ds.field('CNPJ_FUNDO') == '18.255.509/0001-40') | (ds.field('CNPJ_FUNDO') == '11.110.937/0001-36') | (ds.field('CNPJ_FUNDO') == '54.438.899/0001-58') 
| (ds.field('CNPJ_FUNDO') == '39.586.784/0001-17') | (ds.field('CNPJ_FUNDO') == '40.142.095/0001-00')  | (ds.field('CNPJ_FUNDO') == '49.920.190/0001-54'))
dataset = ds.dataset(r'C:\Users\Rodrigo\Python\PythonCVM\CarteirasCVMParquet')
tabelacomp = dataset.to_table(filter=filter_expression)

df_filtrado = tabelacomp.to_pandas()
df_filtrado
df_filtrado.to_excel(r'C:\Users\Rodrigo\Python\PythonCVM\Estudos\teste.xlsx', index=False)

In [ ]:
#filter_expression = (ds.field('CNPJ_FUNDO') == '41.902.409/0001-80')
dataset = ds.dataset(r'C:\Users\Rodrigo\Python\PythonCVM\PLsCVMParquet')
tabelacomp = dataset.to_table(filter=filter_expression)
df_filtrado = tabelacomp.to_pandas()
df_filtrado.to_excel(r'C:\Users\Rodrigo\Python\PythonCVM\Estudos\teste2.xlsx', index=False)
